# Using an autonomous dynamical system to predict cell-cycle dynamics

![image coupling](https://colasdroin.github.io/images/velocity.png)

## Context and output

I have worked on this project from in 2020, during the last year of my PhD. It was a collaboration, in which I was mainly responsible for the optimization of an autonomous dynamical system representing the cell-cycle in gene expression space. This project is actually a sequel to [RNA velocity](https://www.nature.com/articles/s41586-018-0414-6), and I have been actually collaborating with Gioele La Manno for this.

Unfortunately, the project is still very much ongoing, and I am not able to share too many results, neither my own implementation of the model. But I will try to explain briefly what I have been doing.

## Simplified abstract

In this ongoing collaborative study with La Manno’s lab (EPFL), we try to accurately predict the time evolution of single cells in the context of proliferative progress. More specifically, we take interest in the cell-cycle, as it is one of the main drivers of cell-to-cell heterogeneity in gene expression in an otherwise homogeneous cell population. Although various methods have been developed to characterize its progression, they usually rely on a few known markers, and suffer technical batch effects. Single-cell RNA sequencing approaches are also limited as they only provide a static snapshot of the cell expression states. However, taking advantage of the high identifiability of the cell-cycle as a periodic 1D manifold in expression space, we here decided to develop a parametric method based on circular UMAP projection along with a new, cell-consistent version of RNA velocity to directly estimate the cell-cycle speed in any given cell. We validate our methods on several datasets including various cell-types.

## The model

![image vx](https://colasdroin.github.io/images/vx.png)

When the initial RNA velocity paper was published, it was a small revolution in the field of cellular dynamics, as finally allowed for a better comprehension of cell trajectories in gene expression space. The basic idea is that, by comparing the quantities of spliced and unspliced RNA in a cell, we can estimate towards which direction the cell is moving, in expression space, as most RNAs should get spliced eventually, with known splicing rates. In all, for each gene $g$, the basic dynamical equations of RNA velocity, for spliced ($S_g$) and unspliced ($U_g$ )counts, are the following:

$$
    \dot S_g = \beta_g U_g - \gamma_g S_g
$$

and

$$
    \dot U_g = \alpha_g - \beta_g U_g
$$

That is, unspliced RNA is produced at a constant translation rate ($\alpha_g$), while spliced RNA is produced at a constant splicing rate ($\beta_g$) and is degraded at a constant rate ($\gamma_g$). In the original paper, $\beta_g$ was assumed to be $1$, meaning that each gene would actually evolves on a different timescale.

In parallel, in the high-dimensional expression space, we expect some dynamical equation governing the time-evolution of the spliced counts of the form:
$$
    \frac{d\textbf{S}}{dt}=F(\textbf{S}, \textbf{U})
$$
This equation is (wrongly) assumed deterministic, but we'll stick to that for now. Even in this form, it still has a massive number of parameters, and inferring $F$ is not realistic. To make things more simpler, we therefore assume that the gene expression states, noted $y_g(x) = (u_g(x), s_g(x))$ depend on $x\in\cal{M}$, where $\cal{M}$ is a given low-dimensional manifold, and $(u_g(x), s_g(x))$ represent the noiseless version of $S_g$ and $U_g$. We then assume there exists an _autonomous_ equation for $\textbf{x}$:
$$
    \frac{d\textbf{x}}{dt}=V(\textbf{x})~,
$$
which provides a useful low-dimensional approximation of the full dynamics. Thus, in ${\cal M}$, we expect a density $\rho(x)$ and a velocity field $V(x)$. Now, for consistency, we must have:

$$
\frac{ds_g(x(t))}{dt} = (\nabla_x s_g)\cdot V(x(t)) = \beta_g u_g(x(t)) - \gamma_g s_g(x(t))
$$

Now, RNA velocity has shown that many cells were following what seems to look like a circle in expression space:

![image vy](https://colasdroin.github.io/images/vy.png)

We are therefore interested in our model in the case of a cycling manifold. We write a general model for the data (the $c$ index stands for cell, each cell having its own phase $\phi_c$ along the cell-cycle), parameterized by a phase on the manifold $\cal{M}$.

$$
\begin{align}
    & S_{gc}=s_g(\phi_c) +\epsilon_{gc} \\
    &U_{gc}=u_g(\phi_c) +\epsilon_{gc}.
\end{align}
$$
This means that we are assuming that $\cal{M}$ is one dimensional (a circle) and $x \equiv \phi$.

We can now study the dynamics of the system, using the consistency equation above (for each gene):

$$
    \dot s =\beta u -\gamma s=\partial_\phi s(\phi) \omega(\phi)
$$
as our coordinate (phase) follows the equation:

$$
    \dot \phi = \omega(\phi).
$$

From here, the main objective is to optimize $\omega(\phi)$, which predicts the velocity of the cell-cycle, common to all genes. 

Making a few assumptions on the noise distribution and the dynamics of the gene parameters, and a lot of equations, we can show that:

$$
\begin{align}
    &P(U,S| \{\phi\}, \{\nu\}, \omega(\phi), \{\beta\},\{\gamma\})\sim\\
    &\exp{-\frac{1}{2\sigma^2}\sum_{gc}\left(\ln{U_{gc}}-\ln{\frac{1}{\beta_g}\sum_{ff'}\nu_{gf}\left(D_{ff'}\omega(\phi)+\gamma_g \delta_{ff'} \right)\zeta_{f'c}}\right)^2}
\end{align}
$$

Where $\nu$ and $D$ and $\delta$ represent different Fourier harmonics of the signal itself ($S_g$) and of the velocity function, $\omega$.

The question is now, how to optimize this formula? Well, there are many parameters, so this is a not trivial problem, in which some tricks have to be used. But the results are still undisclosed for now.

## Simulations

We can easily generate some data for this problem. We start by building unspliced signals from random Fourier series:

$$u_g(\phi) = \mu_g + \sum_0^n a_n \cos(n\phi) + b_n \sin(n\phi)$$

where $a_n$ and $b_n$ are random Fourier coefficients. This yields signals like this:

![image v1](https://colasdroin.github.io/images/v1.png)

We cant then integrate the unspliced signal according to our model to get the spliced:

$$
s_g(\phi) = \int_0^\phi [\beta_g u_g(\theta) - \gamma_g s_g(\theta)]\frac{1}{\omega}d\theta
$$

Yielding:

![image v2](https://colasdroin.github.io/images/v2.png)


We then extract one cycle at steady-state, and add noise and drop-out to replicate real data:

![image v3](https://colasdroin.github.io/images/v3.png)


As a check, we can look at the low-dimensional t-SNE projection of the generated data:

![image v4](https://colasdroin.github.io/images/v4.png)


We then run the optimization algorithm, yielding nice fits:

![image v5](https://colasdroin.github.io/images/v5.png)

And a pretty accurate velocity function (also generated at the beginning of the procedure):

![image v6](https://colasdroin.github.io/images/v6.png)

The real issue is to have this work on real data... But this is still an unpublished story.